In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Feature Engineering para el Modelo

## Cargar Datos

In [61]:
completo_df = pd.read_csv('../data/raw/df_clean_completo.csv')
print("Datos cargados exitosamente.")
completo_df

Datos cargados exitosamente.


,COD_ALUMNO,SEXO,PER_INGRESO,ESTADO_CIVIL,TIPO_COLEGIO,PTJE_INGRESO,ESTADO,SEM_CURSADOS,CANT_RESERVAS,PER_MATRICULA,COD_CURSO,CURSO,CREDITOS,TIPO_CURSO,COD_PLAN,COD_GRUPO,GRUPO,NOTA,APROBO,HRS_INASISTENCIA,HRS_CURSO,PRCTJE_INASISTENCIA,PONDERADO,CRED_GRADUACION,BECA_VIGENTE,FECHA_NACIMIENTO,POBREZA_RES,POBREZA_PRO,TIPO_CICLO,NIVEL_CURSO,FAMILIA
0,30765,M,2016-02,S,Privada Parroquial,120.00,Regular,12,0,2017-01,CS100,INTRODUCCION DE CIENCIA DE LA COMPUTACION,3,Obligatorio,79,41206,CCOMP2-1,10.81,N,4,4,29,10.82,219,0,1998-09-10,1.43,0.72,Regular,2,CS
1,30765,M,2016-02,S,Privada Parroquial,120.00,Regular,12,0,2017-01,FG104,INTRODUCCION A LA FILOSOFIA,2,Obligatorio,79,41207,CCOMP2-1,9.62,N,9,3,29,10.82,219,0,1998-09-10,1.43,0.72,Regular,2,FG
2,28283,F,2016-01,S,Privada Particular,128.00,Regular,14,0,2017-01,CS112,CIENCIA DE LA COMPUTACION I,5,Obligatorio,79,41216,CCOMP2-1.2,16.21,S,2,8,29,14.27,219,1,1999-03-25,4.70,0.72,Regular,2,CS
3,28283,F,2016-01,S,Privada Particular,128.00,Regular,14,0,2017-01,MA101,MATEMATICA II,4,Obligatorio,79,41217,CCOMP2-1,14.53,S,14,6,29,14.27,219,1,1999-03-25,4.70,0.72,Regular,2,MA
4,28283,F,2016-01,S,Privada Particular,128.00,Regular,14,0,2017-01,CS221,ARQUITECTURA DE COMPUTADORES,3,Obligatorio,79,41210,CCOMP3-1,12.81,S,4,4,29,14.27,219,1,1999-03-25,4.70,0.72,Regular,3,CS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11204,32540,M,2017-01,S,Privada Particular,113.00,Retirado,7,1,2018-01,CS1D2,ESTRUCTURAS DISCRETAS II,4,Obligatorio,79,45525,CCOMP2-1.2,13.30,S,2,6,30,11.65,219,0,1991-02-25,0.35,0.72,Regular,2,CS
11205,32540,M,2017-01,S,Privada Particular,113.00,Retirado,7,1,2018-01,CS100,INTRODUCCION DE CIENCIA DE LA COMPUTACION,3,Obligatorio,79,45527,CCOMP2-1,13.07,S,2,4,30,11.65,219,0,1991-02-25,0.35,0.72,Regular,2,CS
11206,32540,M,2017-01,S,Privada Particular,113.00,Retirado,7,1,2018-01,CS112,CIENCIA DE LA COMPUTACION I,5,Obligatorio,79,45519,CCOMP2-1.2,5.57,N,20,8,30,11.65,219,0,1991-02-25,0.35,0.72,Regular,2,CS
11207,32540,M,2017-01,S,Privada Particular,113.00,Retirado,7,1,2018-01,MA100,MATEMATICA I,5,Obligatorio,79,45645,CIV1-4,11.72,S,2,6,30,11.65,219,0,1991-02-25,0.35,0.72,Regular,1,MA


## Crear Cluster Cursos
Cargar JSON de descripcion de cursos
str -> tf-idf -> SVD(20 components) -> Kmeans

In [77]:
import json
import pandas as pd

# Cargar el JSON
with open('../data/raw/cursos.json', 'r', encoding='utf-8') as archivo:
    cursos_data = json.load(archivo)
# Dataframe
cursos_nlp_df = pd.DataFrame(cursos_data)
cursos_nlp_df['DESCRIPCION_TEXTO'] = cursos_nlp_df['DESCRIPCION'].apply(lambda x: ' '.join(x))
print(cursos_nlp_df[['CODIGO', 'DESCRIPCION_TEXTO']].head())

  CODIGO                                  DESCRIPCION_TEXTO
0  CS111  programming game development algorithms logic ...
1  CS1D1  mathematics logic sets relations functions pro...
2  FG101  communication writing reading speaking listeni...
3  FG102  study techniques learning methods time managem...
4  FG103  university life academic adaptation campus res...


In [78]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=500, stop_words='english')

features_tfidf = vectorizer.fit_transform(cursos_nlp_df['DESCRIPCION_TEXTO'])

print("\nForma de la matriz TF-IDF (cursos, features):")
print(features_tfidf.shape)


Forma de la matriz TF-IDF (cursos, features):
(71, 500)


In [79]:
from sklearn.decomposition import TruncatedSVD

n_componentes = 20
svd = TruncatedSVD(n_components=n_componentes, random_state=42)
features_reducidos = svd.fit_transform(features_tfidf)

print(f"\nForma de la matriz reducida (cursos, componentes):")
print(features_reducidos.shape)


Forma de la matriz reducida (cursos, componentes):
(71, 20)


In [82]:
from sklearn.cluster import KMeans

# Definimos cuántos clusters queremos encontrar (ej. 8)
# (Puedes usar el "Método del Codo" para encontrar el 'k' óptimo)
n_clusters = 8
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)

# Asignamos cada curso a un cluster
clusters = kmeans.fit_predict(features_reducidos)

# ¡Listo! Añadimos el cluster como un nuevo feature a nuestro DataFrame
cursos_nlp_df['CLUSTER_CURSO'] = clusters

print("\nCursos con su cluster asignado:")
print(cursos_nlp_df[['CODIGO', 'CURSO', 'CLUSTER_CURSO']].head())


Cursos con su cluster asignado:
  CODIGO                                 CURSO  CLUSTER_CURSO
0  CS111          Programación de Video Juegos              7
1  CS1D1               Estructuras Discretas I              4
2  FG101                          Comunicación              0
3  FG102               Metodología del Estudio              6
4  FG103  Introducción a la Vida Universitaria              3


In [83]:
cursos_nlp_df[['CODIGO', 'CURSO', 'CLUSTER_CURSO']]

,CODIGO,CURSO,CLUSTER_CURSO
0,CS111,Programación de Video Juegos,7
1,CS1D1,Estructuras Discretas I,4
2,FG101,Comunicación,0
3,FG102,Metodología del Estudio,6
4,FG103,Introducción a la Vida Universitaria,3
...,...,...,...
66,CS404,Proyecto Final de Carrera III,6
67,ET301,Formación de Empresas de Base Tecnológica II,2
68,FG211,Ética Profesional,3
69,FG220,Análisis de la Realidad Peruana,3


In [84]:
for cluster in cursos_nlp_df['CLUSTER_CURSO'].unique():
    print(cursos_nlp_df['CURSO'][cursos_nlp_df['CLUSTER_CURSO'] == cluster].unique())

['Programación de Video Juegos'
 'Introducción de Ciencia de la Computación' 'Ciencia de la Computación I'
 'Arquitectura de Computadores' 'Desarrollo Basado en Plataformas'
 'Sistemas Operativos' 'Computación Gráfica' 'Lenguajes de Programación'
 'Interacción Humano Computador' 'Compiladores'
 'Tópicos en Computación Gráfica']
['Estructuras Discretas I' 'Estructuras Discretas II'
 'Ciencia de la Computación II' 'Algoritmos y Estructuras de Datos'
 'Análisis y Diseño de Algoritmos' 'Programación Competitiva'
 'Estructuras de Datos Avanzadas']
['Comunicación' 'Oratoria' 'Inglés Técnico Profesional']
['Metodología del Estudio' 'Inteligencia Artificial'
 'Metodología de la Investigación en Computación'
 'Proyecto Final de Carrera I' 'Tópicos en Inteligencia Artificial'
 'Proyecto Final de Carrera II' 'Robótica' 'Proyecto Final de Carrera III']
['Introducción a la Vida Universitaria' 'Introducción a la Filosofía'
 'Apreciación Musical' 'Persona, Matrimonio y Familia'
 'Antropología Filosóf

## a